# Preference-Guided Image Captioning Alignment - Exploration Notebook

This notebook demonstrates the key capabilities of our preference-guided image captioning system, including model architecture exploration, training pipeline analysis, and evaluation metrics.

## Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path

# Add src to path
sys.path.append(str(Path().parent / "src"))

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Environment setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## Configuration and Model Setup

In [ ]:
from preference_guided_image_captioning_alignment.utils.config import Config
from preference_guided_image_captioning_alignment.models.model import PreferenceGuidedCaptioningModel
from preference_guided_image_captioning_alignment.data.preprocessing import ImageProcessor, TextProcessor

# Load configuration
config = Config("../configs/default.yaml")
print("Configuration loaded successfully!")
print(f"Model config: {config.get_model_config()}")

In [ ]:
# Create model with smaller dimensions for exploration
model = PreferenceGuidedCaptioningModel(
    vision_model="openai/clip-vit-base-patch32",
    text_model="microsoft/DialoGPT-medium",
    projection_dim=256,  # Smaller for demo
    temperature=0.07,
    dropout=0.1,
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model created successfully!")

## Data Processing Exploration

In [ ]:
# Create data processors
image_processor = ImageProcessor(image_size=224, augment=False)
text_processor = TextProcessor(
    model_name="microsoft/DialoGPT-medium",
    max_length=128,
)

print(f"Image processor: {image_processor.image_size}x{image_processor.image_size}")
print(f"Text processor vocab size: {text_processor.vocab_size:,}")

In [ ]:
# Create sample data
sample_image = Image.new('RGB', (224, 224), color='red')
sample_captions = [
    "A beautiful red sports car on a winding mountain road",
    "A magnificent sunset over the ocean with golden reflections",
    "A cute kitten playing with a ball of yarn",
    "A modern city skyline illuminated at night"
]

# Process image
processed_image = image_processor.process_image(sample_image)
print(f"Processed image shape: {processed_image.shape}")
print(f"Image tensor stats: min={processed_image.min():.3f}, max={processed_image.max():.3f}")

# Process captions
processed_captions = text_processor.encode_batch(sample_captions)
print(f"Processed captions shape: {processed_captions['input_ids'].shape}")
print(f"Caption lengths: {processed_captions['attention_mask'].sum(dim=1)}")

## Model Architecture Analysis

In [ ]:
# Analyze model components
def analyze_model_component(component, name):
    params = sum(p.numel() for p in component.parameters())
    trainable = sum(p.numel() for p in component.parameters() if p.requires_grad)
    print(f"{name}:")
    print(f"  Total params: {params:,}")
    print(f"  Trainable params: {trainable:,}")
    print(f"  Frozen ratio: {(params-trainable)/params:.1%}")
    return params, trainable

vision_params, vision_trainable = analyze_model_component(model.vision_encoder, "Vision Encoder")
text_params, text_trainable = analyze_model_component(model.text_encoder, "Text Encoder")
decoder_params, decoder_trainable = analyze_model_component(model.caption_decoder, "Caption Decoder")

print("\nModel Distribution:")
total = vision_params + text_params + decoder_params
print(f"Vision: {vision_params/total:.1%}")
print(f"Text: {text_params/total:.1%}")
print(f"Decoder: {decoder_params/total:.1%}")

## Forward Pass Exploration

In [ ]:
# Test contrastive mode
batch_size = 4
images = torch.randn(batch_size, 3, 224, 224)
caption_ids = processed_captions['input_ids']
caption_mask = processed_captions['attention_mask']

with torch.no_grad():
    contrastive_outputs = model(
        images=images,
        caption_ids=caption_ids,
        caption_mask=caption_mask,
        mode="contrastive"
    )

print("Contrastive Mode Outputs:")
for key, value in contrastive_outputs.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape}")

# Compute similarity matrix
similarity_matrix = torch.matmul(
    contrastive_outputs['image_embeddings'],
    contrastive_outputs['text_embeddings'].t()
)

print(f"\nSimilarity matrix shape: {similarity_matrix.shape}")
print(f"Similarity range: [{similarity_matrix.min():.3f}, {similarity_matrix.max():.3f}]")

In [ ]:
# Visualize similarity matrix
plt.figure(figsize=(10, 8))
sns.heatmap(
    similarity_matrix.numpy(),
    annot=True,
    fmt='.3f',
    cmap='RdYlBu_r',
    xticklabels=[f'Caption {i+1}' for i in range(batch_size)],
    yticklabels=[f'Image {i+1}' for i in range(batch_size)]
)
plt.title('Image-Caption Similarity Matrix')
plt.xlabel('Captions')
plt.ylabel('Images')
plt.tight_layout()
plt.show()

# Diagonal values (matching pairs)
diagonal_similarities = torch.diagonal(similarity_matrix)
print(f"\nDiagonal similarities (matching pairs): {diagonal_similarities}")
print(f"Mean diagonal similarity: {diagonal_similarities.mean():.3f}")

## Caption Generation Exploration

In [ ]:
# Test caption generation
model.eval()

# Generate captions for sample images
test_images = torch.randn(2, 3, 224, 224)

with torch.no_grad():
    generated_captions = model.generate_captions(
        images=test_images,
        max_length=30,
        num_beams=3,
        temperature=0.8,
        do_sample=True,
        top_p=0.9
    )

print("Generated Captions:")
for i, caption in enumerate(generated_captions):
    print(f"  Image {i+1}: '{caption}'")
    print(f"  Length: {len(caption.split())} words")

## Loss Function Analysis

In [ ]:
from preference_guided_image_captioning_alignment.models.model import ContrastiveLoss, PreferenceLoss

# Test contrastive loss
contrastive_loss_fn = ContrastiveLoss(temperature=0.07)
contrastive_loss = contrastive_loss_fn(
    contrastive_outputs['image_embeddings'],
    contrastive_outputs['text_embeddings']
)

print(f"Contrastive Loss: {contrastive_loss.item():.4f}")

# Test preference loss with dummy data
preference_loss_fn = PreferenceLoss(beta=0.1)

batch_size = 2
seq_len = 20
vocab_size = text_processor.vocab_size

preferred_logits = torch.randn(batch_size, seq_len, vocab_size)
rejected_logits = torch.randn(batch_size, seq_len, vocab_size) * 0.8  # Make slightly worse
labels = torch.randint(0, vocab_size, (batch_size, seq_len))
mask = torch.ones(batch_size, seq_len)

preference_loss = preference_loss_fn(
    preferred_logits, rejected_logits,
    labels, labels,
    mask, mask
)

print(f"Preference Loss: {preference_loss.item():.4f}")

## Temperature Effect Analysis

In [ ]:
# Analyze effect of temperature on contrastive loss
temperatures = np.logspace(-2, 1, 20)  # 0.01 to 10
losses = []

# Create perfect alignment scenario
embeddings = torch.randn(4, 256)
embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=-1)

for temp in temperatures:
    loss_fn = ContrastiveLoss(temperature=temp)
    loss = loss_fn(embeddings, embeddings)  # Perfect alignment
    losses.append(loss.item())

plt.figure(figsize=(10, 6))
plt.semilogx(temperatures, losses, 'b-', linewidth=2, marker='o')
plt.axvline(x=0.07, color='r', linestyle='--', label='Default (0.07)')
plt.xlabel('Temperature')
plt.ylabel('Contrastive Loss')
plt.title('Effect of Temperature on Contrastive Loss (Perfect Alignment)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print(f"Loss at temperature 0.07: {losses[np.argmin(np.abs(temperatures - 0.07))]:.4f}")

## Evaluation Metrics Exploration

In [ ]:
from preference_guided_image_captioning_alignment.evaluation.metrics import CaptioningMetrics

# Create metrics calculator
metrics_calc = CaptioningMetrics(device="cpu")

# Sample predictions and references
predictions = [
    "A red car driving on the street",
    "A beautiful sunset over the ocean",
    "A cat sitting on a chair",
    "Children playing in the park"
]

references = [
    ["A red sports car on the road", "A car driving down the street"],
    ["Beautiful sunset over water", "Golden sunset on the ocean"],
    ["A cat on a chair", "Cat sitting comfortably"],
    ["Kids playing in park", "Children having fun outdoors"]
]

print("Computing evaluation metrics...")
print("Note: Some metrics may take time to load models on first run.")

In [ ]:
# Compute BLEU scores
bleu_scores = metrics_calc.compute_bleu_scores(predictions, references)
print("BLEU Scores:")
for metric, score in bleu_scores.items():
    print(f"  {metric}: {score:.4f}")

# Compute ROUGE scores
rouge_scores = metrics_calc.compute_rouge_scores(
    predictions, [ref[0] for ref in references]
)
print("\nROUGE Scores:")
for metric, score in rouge_scores.items():
    print(f"  {metric}: {score:.4f}")

# Compute diversity metrics
diversity_scores = metrics_calc.compute_diversity_metrics(predictions)
print("\nDiversity Metrics:")
for metric, score in diversity_scores.items():
    print(f"  {metric}: {score:.4f}")

## Training Configuration Analysis

In [ ]:
# Analyze training configuration
stage1_config = config.get_stage1_config()
stage2_config = config.get_stage2_config()
targets = config.get_targets()

print("Training Configuration Analysis:")
print("\nStage 1 (Contrastive Learning):")
print(f"  Batch size: {stage1_config['batch_size']}")
print(f"  Learning rate: {stage1_config['learning_rate']}")
print(f"  Epochs: {stage1_config['num_epochs']}")
print(f"  Warmup steps: {stage1_config['warmup_steps']}")

print("\nStage 2 (Preference Optimization):")
print(f"  Batch size: {stage2_config['batch_size']}")
print(f"  Learning rate: {stage2_config['learning_rate']}")
print(f"  Epochs: {stage2_config['num_epochs']}")
print(f"  DPO beta: {stage2_config['dpo_beta']}")

print("\nTarget Metrics:")
for target, value in targets.items():
    print(f"  {target}: {value}")

In [ ]:
# Visualize training schedule
def simulate_lr_schedule(initial_lr, warmup_steps, total_steps):
    """Simulate cosine learning rate schedule with warmup."""
    steps = np.arange(total_steps)
    lrs = []
    
    for step in steps:
        if step < warmup_steps:
            # Linear warmup
            lr = initial_lr * (step / warmup_steps)
        else:
            # Cosine decay
            progress = (step - warmup_steps) / (total_steps - warmup_steps)
            lr = initial_lr * 0.5 * (1 + np.cos(np.pi * progress))
        lrs.append(lr)
    
    return steps, lrs

# Simulate learning rate schedules for both stages
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Stage 1 schedule
steps1, lrs1 = simulate_lr_schedule(
    stage1_config['learning_rate'],
    stage1_config['warmup_steps'],
    stage1_config['num_epochs'] * 100  # Assume 100 steps per epoch
)

ax1.plot(steps1, lrs1, 'b-', linewidth=2)
ax1.set_xlabel('Training Steps')
ax1.set_ylabel('Learning Rate')
ax1.set_title('Stage 1: Contrastive Learning Schedule')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Stage 2 schedule
steps2, lrs2 = simulate_lr_schedule(
    stage2_config['learning_rate'],
    stage2_config['warmup_steps'],
    stage2_config['num_epochs'] * 50  # Assume 50 steps per epoch
)

ax2.plot(steps2, lrs2, 'r-', linewidth=2)
ax2.set_xlabel('Training Steps')
ax2.set_ylabel('Learning Rate')
ax2.set_title('Stage 2: Preference Optimization Schedule')
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

print(f"Stage 1 LR range: {min(lrs1):.2e} - {max(lrs1):.2e}")
print(f"Stage 2 LR range: {min(lrs2):.2e} - {max(lrs2):.2e}")

## Performance Analysis

In [ ]:
# Analyze memory usage and computational complexity
import time

def measure_forward_pass_time(model, batch_size, num_runs=10):
    """Measure forward pass time for different batch sizes."""
    model.eval()
    times = []
    
    # Warmup
    with torch.no_grad():
        for _ in range(3):
            images = torch.randn(batch_size, 3, 224, 224)
            caption_ids = torch.randint(0, 1000, (batch_size, 32))
            caption_mask = torch.ones(batch_size, 32)
            _ = model(images, caption_ids, caption_mask, mode="contrastive")
    
    # Actual measurement
    with torch.no_grad():
        for _ in range(num_runs):
            images = torch.randn(batch_size, 3, 224, 224)
            caption_ids = torch.randint(0, 1000, (batch_size, 32))
            caption_mask = torch.ones(batch_size, 32)
            
            start_time = time.time()
            _ = model(images, caption_ids, caption_mask, mode="contrastive")
            end_time = time.time()
            
            times.append((end_time - start_time) * 1000)  # Convert to ms
    
    return np.mean(times), np.std(times)

# Test different batch sizes
batch_sizes = [1, 2, 4, 8, 16]
mean_times = []
std_times = []

print("Measuring inference times for different batch sizes...")
for batch_size in batch_sizes:
    mean_time, std_time = measure_forward_pass_time(model, batch_size, num_runs=5)
    mean_times.append(mean_time)
    std_times.append(std_time)
    print(f"Batch size {batch_size:2d}: {mean_time:6.2f} ± {std_time:5.2f} ms")

# Calculate per-sample times
per_sample_times = [mean_time / batch_size for mean_time, batch_size in zip(mean_times, batch_sizes)]

In [ ]:
# Visualize performance analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Total time vs batch size
ax1.errorbar(batch_sizes, mean_times, yerr=std_times, 
             marker='o', linewidth=2, capsize=5)
ax1.set_xlabel('Batch Size')
ax1.set_ylabel('Forward Pass Time (ms)')
ax1.set_title('Total Forward Pass Time vs Batch Size')
ax1.grid(True, alpha=0.3)

# Per-sample time vs batch size
ax2.plot(batch_sizes, per_sample_times, 'ro-', linewidth=2)
ax2.set_xlabel('Batch Size')
ax2.set_ylabel('Per-Sample Time (ms)')
ax2.set_title('Per-Sample Forward Pass Time vs Batch Size')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate efficiency metrics
baseline_time = per_sample_times[0]  # Single sample time
efficiency_gains = [baseline_time / per_sample_time for per_sample_time in per_sample_times]

print("\nBatch Processing Efficiency:")
for batch_size, efficiency in zip(batch_sizes, efficiency_gains):
    print(f"Batch size {batch_size:2d}: {efficiency:.2f}x speedup")

target_latency = targets.get('latency_ms_p95', 150)
max_efficient_batch = max([bs for bs, time in zip(batch_sizes, per_sample_times) 
                          if time <= target_latency])
print(f"\nMaximum efficient batch size (≤{target_latency}ms per sample): {max_efficient_batch}")

## Summary and Insights

In [ ]:
print("=" * 60)
print("PREFERENCE-GUIDED IMAGE CAPTIONING - EXPLORATION SUMMARY")
print("=" * 60)

print("\n🏗️  MODEL ARCHITECTURE:")
print(f"   • Total parameters: {total_params:,}")
print(f"   • Trainable parameters: {trainable_params:,}")
print(f"   • Projection dimension: {model.projection_dim}")
print(f"   • Components: Vision Encoder + Text Encoder + Caption Decoder")

print("\n🔄  TRAINING STRATEGY:")
print(f"   • Stage 1: Contrastive learning (LR: {stage1_config['learning_rate']})")
print(f"   • Stage 2: Preference optimization (LR: {stage2_config['learning_rate']})")
print(f"   • DPO beta: {stage2_config['dpo_beta']}")
print(f"   • Temperature: {model.temperature}")

print("\n📊  TARGET METRICS:")
for target, value in targets.items():
    print(f"   • {target}: {value}")

print("\n⚡  PERFORMANCE INSIGHTS:")
print(f"   • Optimal batch size for latency: {max_efficient_batch}")
print(f"   • Max batching speedup: {max(efficiency_gains):.2f}x")
print(f"   • Single sample latency: ~{per_sample_times[0]:.1f}ms")

print("\n🎯  KEY INNOVATIONS:")
print("   • Dual-stage training: contrastive + preference alignment")
print("   • Cross-modal attention in caption decoder")
print("   • LoRA support for efficient fine-tuning")
print("   • Comprehensive evaluation with 10+ metrics")
print("   • Human preference correlation tracking")

print("\n🔍  TECHNICAL HIGHLIGHTS:")
print("   • CLIP-based vision encoder for robust image understanding")
print("   • GPT-based text encoder and decoder for natural language")
print("   • DPO-style preference optimization")
print("   • Configurable temperature for contrastive learning")
print("   • Mixed precision training support")

print("\n" + "=" * 60)
print("✅ EXPLORATION COMPLETED SUCCESSFULLY!")
print("=" * 60)

## Next Steps

This exploration demonstrates the key capabilities of our preference-guided image captioning system:

1. **Model Architecture**: Multi-component design with vision encoder, text encoder, and caption decoder
2. **Training Pipeline**: Two-stage approach combining contrastive learning and preference optimization
3. **Evaluation Framework**: Comprehensive metrics including BLEU, ROUGE, CIDEr, and preference alignment
4. **Performance Analysis**: Efficient batch processing with configurable latency targets

### Recommended Training Workflow:

```bash
# 1. Train the full pipeline
python scripts/train.py --config configs/default.yaml

# 2. Evaluate the trained model
python scripts/run_evaluation.py --checkpoint outputs/checkpoints/best_model_stage2.pt

# 3. Run specific stages if needed
python scripts/train.py --stage 1  # Contrastive learning only
python scripts/train.py --stage 2  # Preference optimization only
```

### Configuration Tips:

- Adjust `projection_dim` for different performance/quality trade-offs
- Tune `temperature` in contrastive loss for better alignment
- Modify `dpo_beta` to control preference optimization strength
- Use LoRA configuration for memory-efficient fine-tuning

### Expected Results:

With proper training, the model should achieve:
- CIDEr score >= 1.15
- Preference win rate >= 72%
- Human evaluation helpfulness >= 4.2/5
- P95 latency <= 150ms